In [11]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [12]:
import math
import re

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()
test = open('test.txt', 'r').read()

input = puzzle

# Part 1

In [13]:
codes = input.split('\n')

# After experimenting with the problem it turns out that it is important to consider the order that the directions go in, in order to
# make the shortest output possible (e.g. moving ^> compared to >^) - this isn't apparent at first (i.e. moving the first robot
# 2 left and 1 down vs 1 down and 2 left) but as this gets propagated further down the robot chain the difference becomes realised.
# The rules as to efficient movement are not immediately obvious but we can try a search (i.e DFS) to take care of it for us

# First, we need a list of valid moves from each tile to another, both in the numpad and directional graphs
# We have two options: 
# - manually write them all out (kind of doable since there's not significantly many keys
# but unsafe if we accidentally miss one)
# - use a branching algorithm to find all of them for us - much safer option (and good practise!)
#
# We settle for the latter:

numpad_keys = [str(x) for x in range(10)]+['A']
numpad_coords = {
    '7': (0,0), '8': (0,1), '9': (0,2),
    '4': (1,0), '5': (1,1), '6': (1,2),
    '1': (2,0), '2': (2,1), '3': (2,2),
                '0': (3,1), 'A': (3,2)
}

dirpad_keys = ['<','>','^','v','A']
dirpad_coords = {
                '^': (0,1), 'A': (0,2),
    '<': (1,0), 'v': (1,1), '>': (1,2)
}

def good_move(moves):
    '''
    Helper function - reads moves (string of ><^V chars) and returns True if there
    are no 'zigzags' (zigzag e.g. ^>^>^) - optimal moves will only be those that have all
    their vertical and horizontal movement done in one block each (e.g. >>>^^^ (good) compared
    to ^>^>^> (bad)). Returns False otherwise. 
    '''
    last_move = ''
    cannot_see_again = set()
    for move in moves:
        if move in cannot_see_again:
            return False
        if move != last_move:
            cannot_see_again.add(last_move)
        last_move = move
    return True
        

def construct_initial_paths(dirs_needed, cur_pos, robot_type, cur_str='', found_paths=set(), depth=1):
    '''
    Takes in a tuple dirs_needed=(vert_needed, hori_needed) and uses DFS to find
    all possible paths that would apply those directions. Nodes are path segments from start to end tile.
    Takes the start pos as a parameter as well in order to never step onto the empty tile
    robot_type param identifies whether it is the num or dir keypad (only effectively affects
    how we avoid the gap though - we reference the tile symbol etc in the prep to call this function)
    '''
    
    # Had some funky behaviour with lists' memory carrying over between calls,
    # so this is a safety parameter. Shouldn't affect runtime as this is just an
    # initialistion function so not significantly many iterations needed
    if depth==1:
        found_paths.clear()

    # If we have reached the destination tile, return the path we took to get here
    # (to add to the collection of paths that we found). We also disregard all of the
    # paths that contain zigzags - we can be certain that they will not be optimal (added movement
    # for the robots on dirpad)
    if dirs_needed == (0,0) and good_move(cur_str):
        return [cur_str]

    # Get the gap tile that we need to avoid
    match robot_type:
        case 'num': 
            gap = (3,0)
        case 'dir': 
            gap = (0,0)
    
    # Extract number of vertical/horizontal tiles we need to move yet
    vert_needed = dirs_needed[0]
    hori_needed = dirs_needed[1]

    # Branch into various paths. Essentially we either branch into vertical or horizontal movement, with the
    # respective direction based on the polarity of [vert/hori]_needed. Also, we don't branch into a direction if
    # taking that branch means we hover over the gap.

    # Vertical branches
    if vert_needed < 0 and (cur_pos[0]-1, cur_pos[1]) != gap:
        found_paths = found_paths.union(construct_initial_paths((vert_needed+1, hori_needed), (cur_pos[0]-1, cur_pos[1]),
                                        robot_type=robot_type, cur_str=cur_str+'^',found_paths=found_paths, depth=depth+1))
    elif vert_needed > 0 and (cur_pos[0]+1, cur_pos[1]) != gap:
        found_paths = found_paths.union(construct_initial_paths((vert_needed-1, hori_needed), (cur_pos[0]+1, cur_pos[1]),
                                        robot_type=robot_type, cur_str=cur_str+'v',found_paths=found_paths, depth=depth+1))
    # Horizontal branches
    if hori_needed < 0 and (cur_pos[0], cur_pos[1]-1) != gap:
        found_paths =  found_paths.union(construct_initial_paths((vert_needed, hori_needed+1), (cur_pos[0], cur_pos[1]-1),
                                        robot_type=robot_type, cur_str=cur_str+'<',found_paths=found_paths, depth=depth+1))
    elif hori_needed > 0 and (cur_pos[0], cur_pos[1]+1) != gap:
        found_paths = found_paths.union(construct_initial_paths((vert_needed, hori_needed-1), (cur_pos[0], cur_pos[1]+1),
                                        robot_type=robot_type, cur_str=cur_str+'>',found_paths=found_paths, depth=depth+1))
    
    # Finally, return the set of all unique paths we can take to get to the destination tile
    return found_paths

In [14]:
# Initialise numpad moves:

numpad_moves = {}

for start_pos in numpad_keys:
    for end_pos in numpad_keys:
        vert_needed, hori_needed = (numpad_coords[end_pos][0] - numpad_coords[start_pos][0],
                                     numpad_coords[end_pos][1] - numpad_coords[start_pos][1])
        numpad_moves[(start_pos, end_pos)] = construct_initial_paths((vert_needed,hori_needed), numpad_coords[start_pos], 'num')
        if start_pos == end_pos:
            numpad_moves[(start_pos,end_pos)] = set([''])

# Initialise dirpad moves:

dirpad_moves = {}

for start_pos in dirpad_keys:
    for end_pos in dirpad_keys:
        vert_needed, hori_needed = (dirpad_coords[end_pos][0] - dirpad_coords[start_pos][0],
                                     dirpad_coords[end_pos][1] - dirpad_coords[start_pos][1])
        dirpad_moves[(start_pos, end_pos)] = construct_initial_paths((vert_needed,hori_needed), dirpad_coords[start_pos], 'dir')
        if start_pos == end_pos:
            dirpad_moves[(start_pos,end_pos)] = set([''])

#numpad_moves, dirpad_moves

In [15]:
# Now that we have our collection of routes, we can start branching to find the most optimal route.
# Since it is an optimal value we are after, we can prune the branches as well.

# The only route we actually care about is the final long one (two robots later!)
# but this robot's behaviour changes based on all the robots preceeding it...

# This needs to be a DFS specifically - too many permutations for a BFS and we want to be
# able to prune via optimal length

def get_keypad_paths(code, cur_index=0, cur_path='', prev_tile='A', found_paths=set(), keypad_type=''):
    '''
    Takes an input code and returns a set of all paths that could be constructed
    to input that code on the keypad, starting at A

    e.g. 2A (so A -> 2 -> A) returns {^<A>vA, ^<Av>A, <^A>vA, <^Av>A}
    '''
    if not keypad_type:
        if any([x in code for x in ['^','<','>','v']]):
            keypad_type = 'dir'
        else:
            keypad_type = 'num'
    
    match keypad_type:
        case 'num': keypad_moves = numpad_moves
        case 'dir': keypad_moves = dirpad_moves

    if cur_index == len(code):
        return [cur_path]

    tile = code[cur_index]
    moves = keypad_moves[(prev_tile,tile)]
    for move in moves:
        found_paths = found_paths.union(get_keypad_paths(
            code, cur_index=cur_index+1, cur_path=(cur_path+move+'A'), prev_tile=tile, found_paths=found_paths, keypad_type=keypad_type))
        #print(moves, move, found_paths)
    
    return found_paths



def get_optimal_moves(code, depth=0, seen={}):
    '''
    Takes a code as an input, and returns the length of the optimal series of directions/moves that the chain of robots needs
    '''

    # Clear memory when starting
    if depth == 0:
        seen.clear()

    # If we reach the end of the robot chain (i.e. us inputting for the first robot, we just read this
    # as the length)
    if depth == 3:
        return len(code)
    
    # Memory utilisation/pruning - if we've seen this state before, use it and skip the branches
    if (code,depth) in seen:
        #print(f'Seen ({code},{depth}), best length was {seen[(code,depth)]}')
        return seen[(code,depth)]

    # Get all the ways this code can be expressed in the next robot in the chain
    available_paths = get_keypad_paths(code)

    #print(f'depth={depth}, code={code}, paths={available_paths}')
    
    # Set best route at inf initially
    opt_len = math.inf
    for path in available_paths:
        path_len = 0

        # Extract all subpaths - after putting in directions the robot always needs to return to A, so
        # the segments at this particular depth between the A values have the same length when propagated
        # down the chain i.e. this is how we memorise states
        sub_paths = re.findall(r'[\^><v]+A+', path)
        #print(f'branching into {path} at depth={depth+1}. Subpaths are {sub_paths}')
        for sub_path in sub_paths:
            #print(f'branching into subpath {sub_path}')

            # Send this instruction to the next robot in the chain
            opt_subpath_len = get_optimal_moves(sub_path, depth=depth+1)
            #print(f'depth={depth}, code={code}, path={path} subpath={sub_path}, seen={seen}')

            # If we haven't seen this segment at this depth before, memorise the optimal length
            if (code, depth+1) not in seen:
                seen[(sub_path,depth+1)] = opt_subpath_len
            
            # Add this segment's length on to the overall path above's length
            path_len += opt_subpath_len
            #print(f'best path length found for subpath {sub_path} at (depth {depth})={opt_subpath_len}')
        #print(f'opt length for path {path} at depth={depth} is {path_len}\n')

        # If this path length is better than any we have seen out of the possible permutations in available_paths,
        # update the best value
        if path_len < opt_len:
            opt_len = path_len
    
    # Propagate result back up the chain
    return opt_len


# Testing:
#print(get_optimal_moves(codes[0]), '***')
#print(get_optimal_moves('2')) # should output 21 - see below:

# A -> 2 depth = 0
# <^A ||| ^<A depth = 1
# v<<A>^A>A ||| <Av<A>>^A depth = 2
# v<A<AA>>^AvA<^A>AvA^A | v<A<AA>>^AvA^<A>AvA^A | <vA<AA>>^AvA<^A>AvA^A | <vA<AA>>^AvA^<A>AvA^A ||| v<<A>>^Av<A<A>>^AvAA<^A>A | v<<A>>^Av<A<A>>^AvAA^<A>A | v<<A>>^A<vA<A>>^AvAA<^A>A | v<<A>>^A<vA<A>>^AvAA^<A>A
# 21 | 21 | 21 | 21 ||| 25 | 25 | 25 | 25

In [16]:
def complexity(code):
    '''
    Returns the complexity of a code (optimal length of inputs * original digits)
    '''
    orig_digits = int(code[:-1])
    opt_inputs = get_optimal_moves(code)
    return orig_digits*opt_inputs


codes = input.split('\n')

total_complexity = 0

for code in codes:
    total_complexity += complexity(code)
total_complexity



238078

# Part 2

In [24]:
def get_optimal_moves(code, depth=0, seen={}):
    '''
    Takes a code as an input, and returns the length of the optimal series of directions/moves that the chain of robots needs
    '''

    # Clear memory when starting
    if depth == 0:
        seen.clear()

    # If we reach the end of the robot chain (i.e. us inputting for the first robot, we just read this
    # as the length)
    if depth == 26:
        return len(code)
    
    # Memory utilisation/pruning - if we've seen this state before, use it and skip the branches
    if (code,depth) in seen:
        #print(f'Seen ({code},{depth}), best length was {seen[(code,depth)]}')
        return seen[(code,depth)]

    # Get all the ways this code can be expressed in the next robot in the chain
    available_paths = get_keypad_paths(code)

    #print(f'depth={depth}, code={code}, paths={available_paths}')
    
    # Set best route at inf initially
    opt_len = math.inf
    for path in available_paths:
        path_len = 0

        # Extract all subpaths - after putting in directions the robot always needs to return to A, so
        # the segments at this particular depth between the A values have the same length when propagated
        # down the chain i.e. this is how we memorise states
        sub_paths = re.findall(r'[\^><v]+A+', path)
        #print(f'branching into {path} at depth={depth+1}. Subpaths are {sub_paths}')
        for sub_path in sub_paths:
            #print(f'branching into subpath {sub_path}')

            # Send this instruction to the next robot in the chain
            opt_subpath_len = get_optimal_moves(sub_path, depth=depth+1)
            #print(f'depth={depth}, code={code}, path={path} subpath={sub_path}, seen={seen}')

            # If we haven't seen this segment at this depth before, memorise the optimal length
            if (code, depth+1) not in seen:
                seen[(sub_path,depth+1)] = opt_subpath_len
            
            # Add this segment's length on to the overall path above's length
            path_len += opt_subpath_len
            #print(f'best path length found for subpath {sub_path} at (depth {depth})={opt_subpath_len}')
        #print(f'opt length for path {path} at depth={depth} is {path_len}\n')

        # If this path length is better than any we have seen out of the possible permutations in available_paths,
        # update the best value
        if path_len < opt_len:
            opt_len = path_len
    
    # Propagate result back up the chain
    return opt_len

In [25]:
codes = input.split('\n')

total_complexity = 0

for code in codes:
    total_complexity += complexity(code)
total_complexity

293919502998014

In [18]:
True

True